# Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

In [94]:
import requests
import pandas as pd

In [58]:
df = pd.read_excel(r'C:\Users\sergio.carmona\Documents\Certificados Curso DataScience\Videos Curso DataScience\Curso 9. Capstone\Data1.xlsx')
df.head()

,Contenido
0,M1ANot assigned
1,M1BScarborough(Malvern / Rouge)
2,M1CScarborough(Rouge Hill / Port Union / Highl...
3,M1EScarborough(Guildwood / Morningside / West ...
4,M1GScarborough(Woburn)


In [59]:
#Extraer PostalCode
df['PostalCode'] = df['Contenido'].str.extract('(\w{1}\d{1}\w{1})')

In [70]:
#Extraer Borough
df['Borough'] = df['Contenido'].str.extract('(?<=\w{1}\d{1}\w{1})(.*)(?=\()')

In [79]:
df['Temp'] = df['Contenido'].str.extract('(?<=\()(.*)(?=\))')

In [86]:
#Extraer Neighborhood
df['Neighborhood'] = df['Temp'].str.split(' / ')

In [88]:
#Eliminar columnas innecesarias
df = df.drop('Contenido',1).drop('Temp',1)

In [91]:
#Eliminar filas Nan
df = df.dropna()

In [93]:
df.head()

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"[Malvern, Rouge]"
2,M1C,Scarborough,"[Rouge Hill, Port Union, Highland Creek]"
3,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
4,M1G,Scarborough,[Woburn]
5,M1H,Scarborough,[Cedarbrae]


In [95]:
df.shape

(103, 3)